# Attrs
[Attrs](https://www.attrs.org) reduces the amount of boilerplate code when writing Python classes.  

In [1]:
import numpy as np
import typing
import attr
attr.__version__

'19.3.0'

## Example
We want to implement some classes for representing continious and discrete parameters in a parameter space.  
Attrs offers different ways of specifying class attributes. Here, we are going to make use of Python 3.6's type annotations for maximum conciseness.

In [2]:
@attr.s(auto_attribs=True)
class Parameter(object):
    """An abstract parameter"""
    name: str
    unit: str = None

@attr.s(auto_attribs=True, order=False)
class ContinuousParameter(Parameter):
    """Parameter that can take on continous values in the range [lower, upper]."""
    lower: float = 0
    upper: float = 1
    
    def sample(self, n=1):
        return np.random.rand(n) * (self.upper - self.lower) + self.lower 

@attr.s(auto_attribs=True, order=False)
class DiscreteParameter(Parameter):
    """Parameter that can take on discrete values in the given domain."""
    domain: tuple = (0, 1, 2)
        
    def sample(self, n=1):
        return np.random.choice(self.domain, n)

@attr.s(auto_attribs=True, order=False)
class ParameterSpace(object):
    """Space of parameters"""
    dimensions: typing.List[Parameter]
        
    def sample(self, n=1):
        return np.r_[[dim.sample(n) for dim in self.dimensions]].T

What did we get? A set of classes with the init methods correctly set up and informative signatures.

In [3]:
ContinuousParameter?

Init signature:
ContinuousParameter(
    name: str,
    unit: str = None,
    lower: float = 0,
    upper: float = 1,
) -> None
Docstring:      Parameter that can take on continous values in the range [lower, upper].
Type:           type
Subclasses:     


`__repr__` is automatically implemented with a nice result.

In [4]:
c1 = ContinuousParameter(name='weight', lower=1, upper=10)
c1

ContinuousParameter(name='weight', unit=None, lower=1, upper=10)

`__eq__` is automatically implemented with the expected behaviour.

In [5]:
print(c1 == ContinuousParameter(name='weight', lower=1, upper=10))
print(c1 == ContinuousParameter(name='mass', lower=1, upper=10))

True
False


We set `order=False` so comparison operators are not implemented.

In [6]:
c1 > c1

False

And of course we have our implemented sample method.

In [7]:
c1.sample(n=3)

array([6.51636132, 9.14766125, 1.48018686])

Finally, attrs can nicely serialize attributes to JSON.

In [8]:
attr.asdict(c1)

{'name': 'weight', 'unit': None, 'lower': 1, 'upper': 10}

Let's set up a `ParameterSpace`.

In [9]:
ParameterSpace?

Init signature: ParameterSpace(dimensions: List[__main__.Parameter]) -> None
Docstring:      Space of parameters
Type:           type
Subclasses:     


In [10]:
space = ParameterSpace([c1, DiscreteParameter(name='bar', domain=[1, 2, 7])])
space

ParameterSpace(dimensions=[ContinuousParameter(name='weight', unit=None, lower=1, upper=10), DiscreteParameter(name='bar', unit=None, domain=[1, 2, 7])])

In [11]:
space.sample(3)

array([[6.91987245, 1.        ],
       [9.44313443, 2.        ],
       [3.53644497, 7.        ]])

Attrs has a function for serializing attributes. We could use this for serializing our ParameterSpace. However, we'd be missing the information on the parameter type. We could the type an attribute (e.g. type='continuous' for ContinuousParameter), however, this would need to be read-only and not modifiable by the constructor.

In [12]:
attr.asdict(space)

{'dimensions': [{'name': 'weight', 'unit': None, 'lower': 1, 'upper': 10},
  {'name': 'bar', 'unit': None, 'domain': [1, 2, 7]}]}

In [13]:
ParameterSpace([1, 2, 3])

ParameterSpace(dimensions=[1, 2, 3])

## Conclusion
Attrs does reduce the amount of boilerplate. For rather dumb classes this saves a lot of lines, and thus makes the code much more concise, especially in the above form with type annotations and `auto_attribs=True`.    
For more complicated classes, where we have to implement the `__init__` and other dunder methods ourselves, there's a lot less to gain. Given that some knowledge of attrs is needed to understand the code, I'd argue that in these cases it's better to not use it.